In [1]:
!pip install pycuda # install cuda
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=c5e51779317046f62a4dc4cfa1863ab3d8a24bae30b4d2360dc0cc0c8237a2bb
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [2]:
# CUDA kernel
modd = SourceModule("""
__global__ void times_two(const double* A, double* B)
      {
      uint index = blockIdx.x * blockDim.x + threadIdx.x;
      B[index] = A[index] * 2;
      }
  """)

In [3]:
import numpy as np

# Create the input vectors.

a = np.random.randn(10)
a = a.astype(float)

# Allocate the memory on the GPU and copy the vectors.

a_gpu = cuda.mem_alloc(a.size * a.dtype.itemsize)
cuda.memcpy_htod(a_gpu, a)

b = np.empty_like(a)
b_gpu = cuda.mem_alloc(b.size * b.dtype.itemsize)

# Call the CUDA kernel.

vec_add = modd.get_function("times_two")
vec_add(a_gpu, b_gpu, block=(10, 1, 1), grid=(1, 1, 1))


# Copy the result back to the host.

cuda.memcpy_dtoh(b, b_gpu)

a_gpu.free()
b_gpu.free()


# Do same calculation in CPU.

b_cpu = a * 2

  # Verify the result
print(b)
print(b_cpu)
if (b_cpu == b).all():
  print("Both vectors are the same.")
else:
  print("Vectors are not equal, something went wrong.")


[-1.18635089 -1.46675775 -1.73712091 -3.44403885  0.65204713  0.20853664
  1.63180494 -0.88420605 -2.37486045  3.07298196]
[-1.18635089 -1.46675775 -1.73712091 -3.44403885  0.65204713  0.20853664
  1.63180494 -0.88420605 -2.37486045  3.07298196]
Both vectors are the same.
